# Preâmbulo

Imports básicos.

In [0]:
# Basic imports.
import os
import time
import numpy as np
import torch

from torch import nn
from torch import optim

from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from torchvision import models
from torchvision import datasets
from torchvision import transforms

from skimage import io

from sklearn import metrics

from matplotlib import pyplot as plt

%matplotlib inline

cudnn.benchmark = True

# Sintaxe básica do Pytorch

Assim como o NumPy e o MXNet, o Pytorch é uma biblioteca de processamento vetorial/matricial/tensorial. Operações sobre os tensores do Pytorch possuem sintaxe consideravelmente parecida com operações sobre tensores do NumPy e MXNet.

## Casting para o dispositivo correto

Como usaremos processamento vetorial principalmente em GPUs para aprendizado profundo, primeiramente é possível verificar se há uma GPU disponível com o trecho de código abaixo, armazenando os tensores nos dispositivos apropriados.

In [0]:
# Checking if GPU/CUDA is available.
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

## Tensores no Pytorch

Criando tensores novos.

In [0]:
tns = torch.tensor([1, 2, 3, 4, 5, 6])
print(tns)

Reorganizando tensores.

In [0]:
print(tns.view(2, 3))

# Function view() with -1 infers the shape according to the remaining elements.
print(tns.view(3, -1))

Iniciando tensores vazios

In [0]:
tns_0 = torch.zeros(2, 3)
tns_1 = torch.ones(2, 3)

print(tns_0)
print(tns_1)

Iniciando tensores com valores aleatórios.

In [0]:
tns_u = torch.rand(2, 3) # Flat distribution.
print(tns_u)

tns_n = torch.randn(2, 3) # Normal Distribution.
print(tns_n)

tns_perm = torch.randperm(6) # Random permutation of the interval [0, 5].
print(tns_perm)

Operações com tensores.

In [0]:
print(tns_u)
print(tns_n)

tns_sum = tns_u + tns_n
print(tns_sum)

Indexação.

In [0]:
print(tns_sum[1, 1]) # Indexing element.
print(tns_sum[0, :]) # Indexing line.
print(tns_sum[:, 1]) # Indexing column.

Convertendo de tensores do numpy.

In [0]:
np_arr = np.random.randn(2, 3)
print(np_arr, np_arr.dtype)

torch_tns = torch.from_numpy(np_arr)
print(torch_tns)

Concatenando tensores.

In [0]:
print(tns_u)
print(tns_n)

tns_cat = torch.cat((tns_u, tns_n), 0)
print(tns_cat)

Várias outras operações sobre tensores do Pytorch podem ser vistas nos seguintes tutoriais:
1.   https://jhui.github.io/2018/02/09/PyTorch-Basic-operations/
2.   https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

## Treinando uma MLP simples em dados aleatórios

In [0]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

print(x)
print(y)

In [0]:
# Casting tensors to the appropriate device.
x = x.to(device)
y = y.to(device)

print(x)
print(y)

In [0]:
# Printing sizes of tensors.
print(x.size())
print(y.size())

## Definindo arquitetura, loss e otimizador

In [0]:
# Use the nn package to define our model.
model = nn.Sequential(
    nn.Linear(D_in, H),
    nn.ReLU(),
    nn.Linear(H, D_out),
).to(device)

In [0]:
print(model)

Sequential(
  (0): Linear(in_features=1000, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)


In [0]:
# Use the nn package to define our loss function.
loss_fn = nn.MSELoss(reduction='sum').to(device)

In [0]:
# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

## Minimizando o erro entre $f(x)$ e $y$

In [0]:
# Creating list of losses for each epoch.
loss_list = []

# Iterating over epochs.
for epoch in range(500):
    
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    
    if (epoch + 1) % 10 == 0:
        print('Epoch ' + str(epoch + 1) + ': loss = ' + str(loss.item()))
    
    # Updating list of losses for printing.
    loss_list.append(loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

In [0]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

ax.plot(np.asarray(loss_list))

plt.show()